In [1]:
# create dummy dataset
import pandas as pd
import numpy as np 
import torch
from torch import Generator
from torch.utils.data import DataLoader,Dataset, dataloader,random_split

from dataclasses import dataclass

import lightning.pytorch as pl

import dagshub
import mlflow


In [2]:
!wget https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv

--2026-01-21 14:19:39--  https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23105 (23K) [text/plain]
Saving to: ‘diabetes.csv.3’

diabetes.csv.3      100%[===================>]  22.56K  --.-KB/s    in 0s      

2026-01-21 14:19:39 (89.5 MB/s) - ‘diabetes.csv.3’ saved [23105/23105]



In [3]:
from threading import stack_size


url = "https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv"

# Read the data
df = pd.read_csv(url)
df.head()


@dataclass
class DiabeticDataset(Dataset):
    X:torch.Tensor
    y:torch.Tensor

    def __len__(self):
        return len(self.y)

    def __getitem__(self,idx):
        return self.X[idx],self.y[idx]

# normalization
class Normalization_dataset(Dataset):
    def __init__(self, base_dataset, mean, std):
        self.base_dataset = base_dataset
        self.mean = mean
        self.std = std

        # 🔥 preserve indices if base_dataset is a Subset
        if hasattr(base_dataset, "indices"):
            self.indices = base_dataset.indices

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        X, y = self.base_dataset[idx]
        X = (X - self.mean) / (self.std + 1e-8)
        return X, y

class DiabeticDataModule(pl.LightningDataModule):
    def __init__(
        self,
        df,
        batch_size=16,
        train_ratio=0.75,
        seed=40
    ):
        super().__init__()
        self.df=df
        self.batch_size=batch_size
        self.train_ratio=train_ratio
        self.seed=seed

    def setup(self,stage=None):
        X=df.drop(columns='Outcome',axis=1).values
        y=df['Outcome'].values

        # convert into tensor
        X=torch.tensor(X,dtype=torch.float32)
        y=torch.tensor(y,dtype=torch.long)

        full_dataset=DiabeticDataset(X,y)

        train_size = int(self.train_ratio * len(full_dataset))
        test_size = len(full_dataset) - train_size
        generator=torch.Generator().manual_seed(self.seed)

        self.train_ds, self.test_ds = random_split(
            full_dataset,
            [train_size, test_size],
            generator=generator
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            shuffle=True,
            pin_memory=True
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            shuffle=False,
            pin_memory=True
        )

    def normalize_datasets(self):
        X_all = []

        for X, y in self.train_dataloader():
            X_all.append(X.cpu())

        X_all = torch.cat(X_all, dim=0)

        mean = X_all.mean(dim=0)
        std  = X_all.std(dim=0)

        # Wrap datasets
        self.train_ds = Normalization_dataset(self.train_ds, mean, std)
        self.test_ds  = Normalization_dataset(self.test_ds,  mean, std)

        return mean, std




In [4]:
dm = DiabeticDataModule(df=df, seed=36)
dm.setup()

mean, std = dm.normalize_datasets()

train_loader = dm.train_dataloader()
test_loader  = dm.test_dataloader()

print("Mean:", mean)
print("Std:", std)




Mean: tensor([  3.8264, 121.0104,  69.0521,  20.2465,  78.5729,  31.9856,   0.4695,
         33.1042])
Std: tensor([  3.4117,  32.4606,  19.7235,  16.0761, 112.1771,   7.9008,   0.3158,
         11.5159])


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [5]:
# Collect train data
X_train_list = []
y_train_list = []

for x, y in train_loader.dataset:
    X_train_list.append(x)
    y_train_list.append(y)

X_train = torch.stack(X_train_list, dim=0)  # (N_train, num_features)
y_train = torch.tensor(y_train_list)         # (N_train,)

# Collect test data
X_test_list = []
y_test_list = []

for x, y in test_loader.dataset:
    X_test_list.append(x)
    y_test_list.append(y)

X_test = torch.stack(X_test_list, dim=0)   # (N_test, num_features)
y_test = torch.tensor(y_test_list)    

from pathlib import Path
save_dir=Path.cwd().parent/'data'/'splits'
save_dir.mkdir(parents=True,exist_ok=True)

# File path
save_path = save_dir / "diabetes_normalized.pt"

torch.save({
    "X_train": X_train,
    "y_train": y_train,
    "X_test": X_test,
    "y_test": y_test
}, save_path)

In [6]:
# save split data inot csv and store in dvc
train_indices=dm.train_ds.indices
test_indices=dm.test_ds.indices
import pathlib
from pathlib import Path
data_dir=Path.cwd().parent/'data'
# Create 'splits' folder inside 'data' directory
splits_dir = data_dir / 'splits'

splits_dir.mkdir(parents=True, exist_ok=True)

df.iloc[train_indices].to_csv(splits_dir / 'train.csv', index=False)
df.iloc[test_indices].to_csv(splits_dir / 'test.csv', index=False)

# basic algo: logistic_regression

In [7]:
# basic algo: logistic Algorithm

import torch
import torch.nn as nn

class Logistic_RgressionModel(nn.Module):
    def __init__(self, featur_dim):
        super().__init__()
        self.linear=nn.Linear(featur_dim,1)   # single output either 0 or 1
    
    def forward(self,x):
        return self.linear(x)

# setup model , loss and optimizer
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

featur_dim=8

model=Logistic_RgressionModel(featur_dim=featur_dim)

lr=0.001

optimizer=torch.optim.Adam(model.parameters(),lr=lr)

criterion=nn.BCEWithLogitsLoss()


def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct=0
    total=0

    for X, y in loader:
        X = X.to(device)
        y = y.float().unsqueeze(1).to(device)  # (batch, 1)

        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # accuracy
        probs=torch.sigmoid(logits)
        predicts=(probs>0.5).long()
        correct += (predicts == y.long()).sum().item()
        total += y.size(0)


    return total_loss / len(loader),correct/total

def evaluate(model,loader,device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            y = y.to(device)

            logits = model(X)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).long().squeeze(1)

            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total



#







In [8]:
import dagshub
dagshub.init(repo_owner='manikantmnnit', repo_name='diabetes_project', mlflow=True)


mlflow.set_tracking_uri('https://dagshub.com/manikantmnnit/diabetes_project.mlflow')

num_epochs = 50
mlflow.set_experiment("diabetes_logistic_regression")
with mlflow.start_run(run_name='log_reg_baseline'):
    mlflow.log_param("model", "logistic_regression")
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("learning_rate", lr)
    mlflow.log_param('Batch_size',num_epochs)

    for epoch in range(num_epochs):
        train_loss,train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        test_acc   = evaluate(model, test_loader, device)

        # ---- Log metrics per epoch ----
        mlflow.log_metric("train_log_loss", train_loss, step=epoch)
        mlflow.log_metric("train_accuracy", train_acc, step=epoch)
        mlflow.log_metric("test_accuracy", test_acc, step=epoch)

        if (epoch + 1) % 5 == 0:
            print(
                f"Epoch [{epoch+1}/{num_epochs}] | "
                f"Loss: {train_loss:.4f} | "
                f"Train Acc: {train_acc:.4f} | "
                f"Test Acc: {test_acc:.4f}"
            )
    
    # log model
    mlflow.pytorch.log_model(model,artifact_path='model')

Accessing as manikantmnnit

Initialized MLflow to track repo "manikantmnnit/diabetes_project"

Repository manikantmnnit/diabetes_project initialized!

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch [5/50] | Loss: 0.6929 | Train Acc: 0.5347 | Test Acc: 0.5625
Epoch [10/50] | Loss: 0.6232 | Train Acc: 0.6615 | Test Acc: 0.6667
Epoch [15/50] | Loss: 0.5784 | Train Acc: 0.7240 | Test Acc: 0.6979
Epoch [20/50] | Loss: 0.5487 | Train Acc: 0.7431 | Test Acc: 0.7396
Epoch [25/50] | Loss: 0.5281 | Train Acc: 0.7587 | Test Acc: 0.7500
Epoch [30/50] | Loss: 0.5138 | Train Acc: 0.7604 | Test Acc: 0.7604
Epoch [35/50] | Loss: 0.5033 | Train Acc: 0.7656 | Test Acc: 0.7656
Epoch [40/50] | Loss: 0.4960 | Train Acc: 0.7708 | Test Acc: 0.7604
Epoch [45/50] | Loss: 0.4905 | Train Acc: 0.7795 | Test Acc: 0.7760


2026/01/21 14:22:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch [50/50] | Loss: 0.4868 | Train Acc: 0.7778 | Test Acc: 0.7656


2026/01/21 14:22:14 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0+cu128) contains a local version label (+cu128). MLflow logged a pip requirement for this package as 'torch==2.8.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/21 14:22:21 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+cu128) contains a local version label (+cu128). MLflow logged a pip requirement for this package as 'torchvision==0.23.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


🏃 View run log_reg_baseline at: https://dagshub.com/manikantmnnit/diabetes_project.mlflow/#/experiments/0/runs/5ba24e77b2fd4ac0a44c583f9f343a13
🧪 View experiment at: https://dagshub.com/manikantmnnit/diabetes_project.mlflow/#/experiments/0


#  Using AIM to track experiments, ,metrics and models

use lightning module

In [9]:
from aim import Run
from aim.pytorch import track_params_dists

run = Run(experiment='diabetes_logistic_regression')

# log hyperparameters once
run['model'] = 'logistic_regression_pytorch'
run['learning_rate'] = lr
run['epochs'] = num_epochs

run['optimizer'] = 'Adam'
run['loss_fn'] = 'BCEWithLogitsLoss'

for epoch in range(num_epochs):
    # Train & Evaluate
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    test_acc = evaluate(model, test_loader, device)

    # ---- Log metrics per epoch ----
    run.track(train_loss, name="train_log_loss", step=epoch)
    run.track(train_acc, name="train_accuracy", step=epoch)
    run.track(test_acc, name="test_accuracy", step=epoch)

    # ---- Optional: track weight/bias distributions ----
    track_params_dists(model, run=run)

    if (epoch + 1) % 5 == 0:
        print(
            f"Epoch [{epoch+1}/{num_epochs}] | "
            f"Loss: {train_loss:.4f} | "
            f"Train Acc: {train_acc:.4f} | "
            f"Test Acc: {test_acc:.4f}"
        )

# Close Aim run
run.close()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch [5/50] | Loss: 0.4840 | Train Acc: 0.7795 | Test Acc: 0.7760
Epoch [10/50] | Loss: 0.4820 | Train Acc: 0.7778 | Test Acc: 0.7760
Epoch [15/50] | Loss: 0.4804 | Train Acc: 0.7778 | Test Acc: 0.7708
Epoch [20/50] | Loss: 0.4793 | Train Acc: 0.7778 | Test Acc: 0.7760
Epoch [25/50] | Loss: 0.4786 | Train Acc: 0.7778 | Test Acc: 0.7865
Epoch [30/50] | Loss: 0.4779 | Train Acc: 0.7778 | Test Acc: 0.7865
Epoch [35/50] | Loss: 0.4775 | Train Acc: 0.7778 | Test Acc: 0.7865
Epoch [40/50] | Loss: 0.4772 | Train Acc: 0.7778 | Test Acc: 0.7865
Epoch [45/50] | Loss: 0.4769 | Train Acc: 0.7778 | Test Acc: 0.7865
Epoch [50/50] | Loss: 0.4767 | Train Acc: 0.7812 | Test Acc: 0.7865


In [15]:
# Log Metrics from LightningModule



class LogisticRegressionLightningModel(pl.LightningModule):
    def __init__(self,input_dim,lr=0.001):
        super().__init__()
        self.linear=nn.Linear(input_dim,1)
        self.lr=lr
        
        self.criterion=torch.nn.BCEWithLogitsLoss()

        # saved hyperparameters
        self.save_hyperparameters()

    def forward(self,x):
        x = x.view(x.size(0), -1)        # ensure batch dimension
        return self.linear(x) 

    def training_step(self,batch,batch_idx):
        X,y=batch
        y=y.float().unsqueeze(1)
        # output by model
        logits=self(X)
        loss=self.criterion(logits,y)

        probs=torch.sigmoid(logits)
        preds=(probs>0.5).float()
        acc=(preds==y).float().mean()

        self.log('train_loss',loss,prog_bar=True)
        self.log('train_accuracy',acc,prog_bar=True)
        return loss

    def test_step(self,batch,batch_idx):
        X,y=batch
        y=y.float().unsqueeze(1)
        # output by model
        logits=self(X)
        loss=self.criterion(logits,y)

        probs=torch.sigmoid(logits)
        preds=(probs>0.5).float()
        acc=(preds==y).float().mean()

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    














In [17]:
# dm = DiabeticDataModule(df=df, batch_size=32)
# model = LogisticRegressionLightningModel(input_dim=8, lr=0.01)

 # set up AIm and mlflow loggers in lightning
from lightning.pytorch.loggers import MLFlowLogger
mlflow_logger=MLFlowLogger(
    experiment_name="diabetes_logistic_regression",
    tracking_uri="file:./mlruns" 
)

trainer = pl.Trainer(
    max_epochs=num_epochs,
    logger=[mlflow_logger],
    accelerator="auto",
    devices=1
)

dm = DiabeticDataModule(df=df, batch_size=32)
model = LogisticRegressionLightningModel(input_dim=8, lr=0.01)

trainer.fit(model, datamodule=dm)
trainer.test(model, datamodule=dm)


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | linear    | Linear            | 9      | train
1 | criterion | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------
9         Trainable params
0         Non-trainable params
9         Total params
0.000     Total estimated model params size (MB)
2         Modules in train mode
0         Modules in eval mode
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.6822916865348816     │
│         val_loss          │    0.6908219456672668     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.6908219456672668, 'val_acc': 0.6822916865348816}]